In [2]:
# 🔧 Importa as bibliotecas necessárias
import pandas as pd                                # Para trabalhar com tabelas
import requests                                    # Para fazer chamadas à API
import time                                        # Para controlar o tempo entre chamadas
from urllib.parse import quote_plus               # Para codificar nomes em URLs
from requests.exceptions import RequestException  # Para capturar erros da API
from unidecode import unidecode                   # Para remover acentos das cidades

# 📁 1. Leitura do arquivo CSV com os dados dos países
df = pd.read_csv("df_fato_pais.csv")  # Certifique-se de que o arquivo está no mesmo diretório do script

# 🧹 2. Filtra os dados válidos: capital e código do país (sigla de 2 letras como BR, US, etc.)
cidades_paises = [
    (row['capital'], row['nome_resumido'])
    for _, row in df.iterrows()
    if (
        pd.notna(row['capital']) and                      # Capital não pode ser ausente
        pd.notna(row['nome_resumido']) and                # Código do país também não
        row['capital'].strip().lower() != 'não informado' and
        row['capital'].strip() != ''                      # Capital não pode estar vazia
    )
]

# 🧼 3. Remove duplicatas, caso algum país tenha aparecido mais de uma vez
cidades_paises = list(set(cidades_paises))

# 🌡️ 4. Lista onde vamos guardar os dados climáticos coletados
dados_clima = []

# ❌ 5. Lista de capitais que falharam ao buscar o clima (útil para revisar depois)
capitais_falharam = []

# 🔑 6. Sua chave da API da OpenWeatherMap (você precisa se cadastrar no site deles para conseguir)
api_key = "e0b696d684ea5b8a15f7f141ca979bc3"  # <<<<<<<<<<<<<<< Substitua isso pela sua chave real

# 🔁 7. Loop para buscar o clima de cada cidade
for cidade_original, codigo_pais in cidades_paises:

    # 🔤 Remove acentos da cidade (ex: "Brasília" → "Brasilia")
    cidade_sem_acento = unidecode(cidade_original)

    # 🌐 Codifica para URL (ex: "San Jose" → "San+Jose")
    cidade_codificada = quote_plus(cidade_sem_acento)

    try:
        # 🔗 Monta os parâmetros para chamada da API
        url = "https://api.openweathermap.org/data/2.5/weather"
        params = {
            "q": f"{cidade_codificada},{codigo_pais}",  # Ex: "Brasilia,BR"
            "appid": api_key,
            "units": "metric",                          # Para temperatura em graus Celsius
            "lang": "pt_br"                             # Retorno em português
        }

        # 🌐 Chama a API com timeout de 10 segundos
        resposta = requests.get(url, params=params, timeout=10)
        resposta.raise_for_status()  # Se a resposta for erro (ex: 404), levanta exceção

        # 🔄 Converte a resposta JSON da API para um dicionário Python
        clima_info = resposta.json()

        # ✅ Salva os dados relevantes da resposta na lista
        dados_clima.append({
            "cidade": cidade_original,
            "codigo_pais": clima_info['sys']['country'],
            "temperatura": clima_info['main']['temp'],
            "sensacao_termica": clima_info['main']['feels_like'],
            "temperatura_min": clima_info['main']['temp_min'],
            "temperatura_max": clima_info['main']['temp_max'],
            "umidade": clima_info['main']['humidity'],
            "descricao_clima": clima_info['weather'][0]['description'],
            "horario_medicao": clima_info['dt']  # Timestamp do horário da medição
        })

    except RequestException as e:
        # ⚠️ Caso ocorra erro da API (ex: cidade não encontrada)
        print(f"[ERRO API] {cidade_original}, {codigo_pais}: {e}")
        capitais_falharam.append((cidade_original, codigo_pais))

    except Exception as e:
        # ⚠️ Qualquer outro erro inesperado
        print(f"[ERRO INESPERADO] {cidade_original}, {codigo_pais}: {e}")
        capitais_falharam.append((cidade_original, codigo_pais))

    # ⏱️ Espera 1 segundo para não ultrapassar limite da API
    time.sleep(1)

# 📊 8. Converte a lista de dados em uma tabela
df_clima = pd.DataFrame(dados_clima)

# 🖨️ 9. Mostra os dados climáticos coletados
print("\n📋 Dados coletados:")
print(df_clima)

# ❗ 10. Exibe as capitais que não foram encontradas
print("\n❌ Capitais com erro:")
for cidade, pais in capitais_falharam:
    print(f" - {cidade} ({pais})")

# 💾 11. (Opcional) Salva os dados em arquivos CSV
df_clima.to_csv("df_clima_capitais.csv", index=False)
pd.DataFrame(capitais_falharam, columns=["cidade", "codigo_pais"]).to_csv("df_falhas_clima.csv", index=False)


[ERRO API] West Island, CC: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=West%2BIsland%2CCC&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] Washington, D.C., US: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Washington%252C%2BD.C.%2CUS&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] City of San Marino, SM: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=City%2Bof%2BSan%2BMarino%2CSM&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] Port of Spain, TT: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Port%2Bof%2BSpain%2CTT&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] The Valley, AI: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=The%2BValley%2CAI&appid=e0b696d684ea5b8a15f7f141ca979bc3&uni

In [3]:

display(df_clima)


,cidade,codigo_pais,temperatura,sensacao_termica,temperatura_min,temperatura_max,umidade,descricao_clima,horario_medicao
0,Yerevan,AM,17.09,15.95,17.09,17.09,42,céu limpo,1758730814
1,Dili,TL,26.64,26.64,26.64,26.64,47,nublado,1758730899
2,Budapest,HU,18.15,17.93,17.11,19.36,73,céu limpo,1758730769
3,Tallinn,EE,11.19,9.67,11.19,11.19,50,algumas nuvens,1758730902
4,Beirut,LB,26.41,26.41,24.10,26.41,65,nuvens dispersas,1758730745
...,...,...,...,...,...,...,...,...,...
191,Dublin,IE,14.66,14.22,13.82,15.21,78,nublado,1758730728
192,Kabul,AF,20.34,19.03,20.34,20.34,23,céu limpo,1758730772
193,Jakarta,ID,28.77,31.09,25.50,28.94,63,nublado,1758730544
194,Basseterre,KN,31.21,38.17,31.21,31.21,70,chuva leve,1758730776


In [4]:
from pandasql import sqldf

# Define uma função para executar consultas SQL
pysqldf = lambda q: sqldf(q, globals())

In [5]:
consultaSQL = """

SELECT * FROM df_clima 
--where codigo_pais = "AR"

"""

resultado = pysqldf(consultaSQL)
display(resultado)

,cidade,codigo_pais,temperatura,sensacao_termica,temperatura_min,temperatura_max,umidade,descricao_clima,horario_medicao
0,Yerevan,AM,17.09,15.95,17.09,17.09,42,céu limpo,1758730814
1,Dili,TL,26.64,26.64,26.64,26.64,47,nublado,1758730899
2,Budapest,HU,18.15,17.93,17.11,19.36,73,céu limpo,1758730769
3,Tallinn,EE,11.19,9.67,11.19,11.19,50,algumas nuvens,1758730902
4,Beirut,LB,26.41,26.41,24.10,26.41,65,nuvens dispersas,1758730745
...,...,...,...,...,...,...,...,...,...
191,Dublin,IE,14.66,14.22,13.82,15.21,78,nublado,1758730728
192,Kabul,AF,20.34,19.03,20.34,20.34,23,céu limpo,1758730772
193,Jakarta,ID,28.77,31.09,25.50,28.94,63,nublado,1758730544
194,Basseterre,KN,31.21,38.17,31.21,31.21,70,chuva leve,1758730776


In [6]:
resultado['horario_medicao'] = pd.to_datetime(resultado['horario_medicao'], unit='s').dt.strftime('%d/%m/%Y %H:%M:%S')

In [7]:
display(resultado)

,cidade,codigo_pais,temperatura,sensacao_termica,temperatura_min,temperatura_max,umidade,descricao_clima,horario_medicao
0,Yerevan,AM,17.09,15.95,17.09,17.09,42,céu limpo,24/09/2025 16:20:14
1,Dili,TL,26.64,26.64,26.64,26.64,47,nublado,24/09/2025 16:21:39
2,Budapest,HU,18.15,17.93,17.11,19.36,73,céu limpo,24/09/2025 16:19:29
3,Tallinn,EE,11.19,9.67,11.19,11.19,50,algumas nuvens,24/09/2025 16:21:42
4,Beirut,LB,26.41,26.41,24.10,26.41,65,nuvens dispersas,24/09/2025 16:19:05
...,...,...,...,...,...,...,...,...,...
191,Dublin,IE,14.66,14.22,13.82,15.21,78,nublado,24/09/2025 16:18:48
192,Kabul,AF,20.34,19.03,20.34,20.34,23,céu limpo,24/09/2025 16:19:32
193,Jakarta,ID,28.77,31.09,25.50,28.94,63,nublado,24/09/2025 16:15:44
194,Basseterre,KN,31.21,38.17,31.21,31.21,70,chuva leve,24/09/2025 16:19:36


In [8]:
df_export = pd.DataFrame(resultado)  # Dados dim pais
df_export.to_csv("df_dim_tempo_capital.csv", index=False)

In [9]:
# ==========================================================
# Importando bibliotecas necessárias
# ==========================================================
from sqlalchemy import create_engine

# ==========================================================
# Definindo as credenciais do banco (Render)
# ==========================================================
host = "dpg-d39eabemcj7s73f0vcv0-a.oregon-postgres.render.com"
port = "5432"
database = "projeto_paises"
user = "user"
password = "QP7z5ILeinmGr76fDpSeNIiqLOeXL4WE"

# ==========================================================
# Criando a conexão com o PostgreSQL usando SQLAlchemy
# ==========================================================
engine = create_engine(
    f"postgresql://{user}:{password}@{host}:{port}/{database}"
)


# ==========================================================
# Salvando no banco (tabela: paises)
# if_exists:
#   - 'replace' -> apaga e recria a tabela
#   - 'append' -> adiciona os dados
# ==========================================================
df_export.to_sql("tbl_clima_capital_pais", engine, if_exists="replace", index=False)

print("✅ Dados salvos no banco Render com sucesso!")


✅ Dados salvos no banco Render com sucesso!


In [10]:
import psycopg2

# Parâmetros de conexão
dbname   = 'projeto_paises'
user     = 'postgres'
password = '252500'
host     = 'localhost'
port     = '5432' 

# Criar uma conexão
conn = psycopg2.connect(dbname=dbname,
                        user=user,
                        password=password,
                        host=host,
                        port=port)


cur = conn.cursor()# Criar um cursor  deixa manipular os dados
#cur.execute("DELETE FROM TBL_CLIMA_CAPITAL_PAIS")

for indice, colunas_df in resultado.iterrows():
        cur.execute('''                                
                insert into TBL_CLIMA_CAPITAL_PAIS (   
                 cidade,
                 codigo_pais,
                 temperatura,
                 sensacao_termica,
                 temperatura_min,
                 temperatura_max,
                 umidade,
                 descricao_clima,
                 horario_medicao)
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
                        ''',
        (
        colunas_df["cidade"],
        colunas_df["codigo_pais"],
        colunas_df["temperatura"],
        colunas_df["sensacao_termica"],
        colunas_df["temperatura_min"],
        colunas_df["temperatura_max"],
        colunas_df["umidade"],
        colunas_df["descricao_clima"],
        colunas_df["horario_medicao"]
        )
        )

#comandos sql transct-sql
conn.commit() # validar alterações que agente fez  e subir para o banco de dados 

# Fechar o cursor e a conexão
cur.close()
conn.close()